<a href="https://colab.research.google.com/github/Lokkamithran/FYP/blob/main/ML%20Models/Naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import f1_score
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import tensorflow as tf


from imblearn.over_sampling import SMOTE

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df = pd.read_csv("drive/MyDrive/fyp-batch33/dataset/Mubys_part_preprocessed_mlmodel.csv", sep="`")
df = pd.read_csv("drive/MyDrive/fyp-batch33/dataset/dataset_offensive.csv", sep="`")
#df_notoffensive = df[df["Fine Tag"]=="Not_offensive"]
#print(df)

In [ ]:
df.head()

,Fine Tag,Coarse Tag,Comment
0,Offensive_others,Offensive_Targeted_Insult_Individual,yaaruda moonsooru vaayana joke ku edukkirathu ...
1,Offensive_others,Offensive_Targeted_Insult_Group,.... ?
2,Offensive_sexuality,Offensive_Untargeted,are u a virgin?
3,Offensive_others,Offensive_Untargeted,fuck off disliker salaaaa power ajith kumar
4,Offensive_others,Offensive_Targeted_Insult_Group,ennada mairu madiri iruku trailer. kadavule


In [ ]:
#seperation comments for classification
df_notoffensive = df[df["Fine Tag"] == "Not_offensive"]
df_offensive = df[df["Fine Tag"] != "Not_offensive"]

In [ ]:
#print(df_offensive)
df_offensive.to_csv('/content/drive/MyDrive/fyp-batch33/dataset/dataset_offensive.csv', index=False,sep="`")
df_notoffensive.to_csv('/content/drive/MyDrive/fyp-batch33/dataset/dataset_notoffensive.csv', index=False,sep="`")

In [ ]:
df

,Fine Tag,Coarse Tag,Comment
0,Offensive_others,Offensive_Targeted_Insult_Individual,yaaruda moonsooru vaayana joke ku edukkirathu ...
1,Offensive_others,Offensive_Targeted_Insult_Group,.... ?
2,Offensive_sexuality,Offensive_Untargeted,are u a virgin?
3,Offensive_others,Offensive_Untargeted,fuck off disliker salaaaa power ajith kumar
4,Offensive_others,Offensive_Targeted_Insult_Group,ennada mairu madiri iruku trailer. kadavule
...,...,...,...
5323,Offensive_others,Offensive_Targeted_Insult_Individual,paatu super but ajith paadurathu semma asingam...
5324,Offensive_others,Offensive_Untargeted,song keka nala iruthuchi bt video mokka
5325,Offensive_others,Offensive_Targeted_Insult_Individual,laugh riot from 0.24 dai kelava
5326,Offensive_others,Offensive_Untargeted,"cofee splllunga ithu oru comedy pundayan , kee..."


In [ ]:
df['Comment'].dropna(inplace=True)
df['Fine Tag'].dropna(inplace=True)
df.dropna(subset=['Comment', 'Fine Tag'], inplace=True)

In [ ]:
print(df['Fine Tag'].value_counts())

Fine Tag
Offensive_others         4484
Offensive_women           301
Offensive_caste           256
Offensive_race            116
Offensive_sexuality        59
Offensive_handicapped      37
Offensive_religion         28
Name: count, dtype: int64


In [ ]:
x = df["Comment"].dropna().values
y = df["Fine Tag"].dropna().values



Tfidf_vect = TfidfVectorizer(max_features=17293)
fit = Tfidf_vect.fit(x)
x = fit.transform(x)


encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(encoder.classes_)
print(encoder.transform(encoder.classes_))


#x = pd.DataFrame(x)
#y=pd.DataFrame(y)

#x = x.sample(n=y.shape[0], random_state=42)
#y = y.sample(n=x.shape[0], random_state=42)

#x = x.to_sparse()

#x = x.toarray()
#y = y.toarray()

oversample = SMOTE(sampling_strategy = 'not majority')
x, y = oversample.fit_resample(x, y)

unique, frequency = np.unique(y, return_counts = True)
print(unique)
print(frequency)


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

['Offensive_caste' 'Offensive_handicapped' 'Offensive_others'
 'Offensive_race' 'Offensive_religion' 'Offensive_sexuality'
 'Offensive_women']
[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]
[4484 4484 4484 4484 4484 4484 4484]


In [ ]:
import pickle
pickle.dump(fit, open("bayes_tfidf.pickle", "wb"))

In [ ]:
model=make_pipeline(MultinomialNB())

model.fit(x_train,y_train)

Pipeline(steps=[('multinomialnb', MultinomialNB())])

In [ ]:
pickle.dump(model, open('naive_bayes_model.pkl', 'wb'))

In [ ]:
y_predicted=model.predict(x_test)

In [ ]:
print(y_predicted)

[6 5 3 ... 5 5 1]


In [ ]:
print("Accuracy: ", accuracy_score(y_test, y_predicted))

Accuracy:  0.8965700329191887


In [ ]:
f1 = f1_score(y_test, y_predicted, average='weighted')
print(f1)

0.8969135341017842


In [ ]:
precision_score=precision_score(y_test, y_predicted,average='weighted')
recall = recall_score(y_test, y_predicted,average='weighted')

In [ ]:
print(precision_score)
print(recall)

0.9147249777526559
0.8965700329191887


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sample_text= "Matha naatu kaarangala india kulla vidave kudathu"
test_x = Tfidf_vect.fit_transform([sample_text])
test_emb_doc = pad_sequences(test_x.toarray(), padding="pre", maxlen=17293)
y_final = model.predict(test_emb_doc)


In [ ]:
print(y_final)

[5]


In [ ]:
def mapping(prediction):
    # if prediction == 0:
    #     output = "not offensive."
    if prediction == 0:
        output = "offensive to other castes."
    elif prediction == 1:
        output = "offensive to handicapped people."
    elif prediction == 2:
        output = "offensive to others."
    elif prediction == 3:
        output = "racially offensive."
    elif prediction == 4:
        output = "offensive to other religions."
    elif prediction == 5:
        output = "offensive to people of the LGBTQIA+ community."
    else:
        output = "offensive to women."
    return output

In [ ]:
mapping(y_final)

'offensive to people of the LGBTQIA+ community.'